In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from functools import wraps
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier
from scipy.stats import binom


In [ ]:
"""
Tratamento e Seleção de variaveis
PCA,SelectKBest, f_classif,accuracy_score, classification_report,GridSearchCV, hiper parametros

#https://scikit-learn.org/stable/modules/feature_selection.html
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

"""

In [118]:
import warnings
warnings.simplefilter(action='ignore')

In [119]:
path=r"D:\appalgoritmos\excelfiles\Customer-Churn.csv"

In [120]:

def binomial_probability_decorator(func):
    def wrapper(df, *args, **kwargs):
        result = func(df, *args, **kwargs)
        
        total_rows = len(result)
        positive_count = (result == 'Sim').sum()
        
        given_probability = positive_count / total_rows
        calculated_probability = binom.pmf(positive_count, total_rows, given_probability)
        
        print("Resultado:", result)
        print("Probabilidade fornecida:", given_probability)
        print("Probabilidade calculada:", calculated_probability)
        
        return result, given_probability, calculated_probability
    return wrapper



In [121]:

def optimize_xgboost_decorator(func):
    def wrapper(X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
        
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_resampled)
    
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        
        classifier = XGBClassifier(random_state=42)
        
        grid_search = GridSearchCV(classifier, param_grid, cv=3, verbose=1, n_jobs=-1)
        grid_search.fit(X_train_resampled, y_train_encoded)
        
        y_probs = grid_search.predict_proba(X_test)
        y_probs_positive = y_probs[:, 1]
        
        y_pred_encoded = grid_search.predict(X_test)
        y_pred = label_encoder.inverse_transform(y_pred_encoded)
        
        accuracy = accuracy_score(y_test, y_pred)
        print("Precisão no conjunto de teste:", accuracy)
        
        print("\nRelatório de Classificação:")
        print(classification_report(y_test, y_pred))
        
        print("\nExemplos de resultados reais, previstos e probabilidades:")
        results_df = pd.DataFrame({'Real': y_test, 'Previsto': y_pred, 'Probabilidade_Pos': y_probs_positive})
        print(results_df.head(3))
        
        return grid_search
    return wrapper



In [122]:

def optimize_mlp_decorator_keras(func):
    def wrapper(X, y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
        
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_resampled)
        
      
        model = Sequential([
            Dense(64, activation='relu', input_shape=(X_train_resampled.shape[1],)),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        
     
        model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
        
    
        model.fit(X_train_resampled, y_train_encoded, epochs=10, batch_size=64, verbose=1)
        
 
        y_probs = model.predict(X_test)
        y_pred = np.round(y_probs) 
        y_pred_encoded = label_encoder.inverse_transform(y_pred.flatten().astype(int)) 
        
        accuracy = accuracy_score(y_test, y_pred_encoded)
        print("Precisão no conjunto de teste:", accuracy)
        
        print("\nRelatório de Classificação:")
        print(classification_report(y_test, y_pred_encoded))
        
        print("\nExemplos de resultados reais, previstos e probabilidades:")
        results_df = pd.DataFrame({'Real': y_test, 'Previsto': y_pred_encoded, 'Probabilidades': y_probs.flatten()})
        print(results_df.head(3))
        
        return model
    return wrapper



In [123]:

def optimize_mlp_decorator(func):
    def wrapper(X, y):
     
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
 
        ros = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
        
      
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_resampled)
        
       
        param_grid = {
            'hidden_layer_sizes': [(50, 50), (100, 100), (50, 100, 50)],
            'activation': ['relu', 'tanh'],
            'solver': ['adam'],
            'alpha': [0.0001, 0.001, 0.01],
            'max_iter': [200, 300, 400]
        }
        
      
        classifier = MLPClassifier(random_state=42)
        
  
        grid_search = GridSearchCV(classifier, param_grid, cv=3, verbose=1, n_jobs=-1)
        grid_search.fit(X_train_resampled, y_train_encoded)
        
    
        y_pred_encoded = grid_search.predict(X_test)
        y_pred = label_encoder.inverse_transform(y_pred_encoded)
        
      
        y_probs = grid_search.predict_proba(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        print("Precisão no conjunto de teste:", accuracy)
        
        print("\nRelatório de Classificação:")
        print(classification_report(y_test, y_pred))
        
        print("\nExemplos de resultados reais, previstos e probabilidades:")
        results_df = pd.DataFrame({'Real': y_test, 'Previsto': y_pred, 'Probabilidades': y_probs.tolist()})
        print(results_df.head(3))
        
        return grid_search
    return wrapper



def preprocess_data_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        df = func(*args, **kwargs)
        
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
        df_filtered = df.apply(lambda col: col.fillna(0) if col.name in categorical_columns else col)
        
        # OneHotEncoder
        encoder = OneHotEncoder(sparse=False)
        encoded_data = encoder.fit_transform(df_filtered[categorical_columns])
        encoded_columns = encoder.get_feature_names_out(input_features=categorical_columns)
        encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)
        
  
        pca = PCA(n_components=2)  
        pca_data = pca.fit_transform(encoded_df)
        
   
        selector = SelectKBest(f_classif, k=1) 
        selected_data = selector.fit_transform(pca_data, df_filtered['Churn']) 
   
        X = selected_data
        y = df_filtered['Churn']
        
        return X, y
    return wrapper





In [124]:

dados = pd.read_csv(
    path,sep=",",encoding="latin-1")
X, y = database_preprocessing(dados)

In [125]:
dados = pd.read_csv(
    path,sep=",",encoding="latin-1")
dados.head(3)

,Maior65Anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,VariasLinhasTelefonicas,ServicoDeInternet,SegurancaOnline,BackupOnline,SeguroNoDispositivo,SuporteTecnico,TVaCabo,StreamingDeFilmes,TipoDeContrato,PagamentoOnline,FormaDePagamento,ContaMensal,Churn
0,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.85,Nao
1,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,ChequePapel,56.95,Nao
2,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,53.85,Sim


In [126]:
#calcula probabilidade binomial
@binomial_probability_decorator
def simulate_prediction(df):
    return np.random.choice(['Sim', 'Nao'], size=len(df), p=[0.3, 0.7])


In [127]:
@preprocess_data_decorator
def database_preprocessing(df):
    return df

In [128]:
#MLP
@optimize_mlp_decorator
def churn_prediction(X, y):
    return X, y


model = churn_prediction(X, y)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
Precisão no conjunto de teste: 0.6905606813342796

Relatório de Classificação:
              precision    recall  f1-score   support

         Nao       0.87      0.68      0.76      1036
         Sim       0.45      0.71      0.55       373

    accuracy                           0.69      1409
   macro avg       0.66      0.70      0.66      1409
weighted avg       0.76      0.69      0.71      1409


Exemplos de resultados reais, previstos e probabilidades:
     Real Previsto                             Probabilidades
185   Sim      Sim   [0.3949991812494841, 0.6050008187505159]
2715  Nao      Sim   [0.4330328737830097, 0.5669671262169903]
3825  Nao      Nao  [0.5904001709127282, 0.40959982908727177]


In [137]:
result, given_prob, calc_prob = simulate_prediction(dados)

Resultado: ['Sim' 'Nao' 'Nao' ... 'Sim' 'Sim' 'Nao']
Probabilidade fornecida: 0.30356382223484313
Probabilidade calculada: 0.010338234674844607


In [129]:
#TensorFlow/Keras

@optimize_mlp_decorator_keras
def churn_prediction_keras(X, y):
    return X, y


model = churn_prediction_keras(X, y)

Epoch 1/10
130/130 [==============================] - 1s 1ms/step - loss: 0.6101 - accuracy: 0.6625
Epoch 2/10
130/130 [==============================] - 0s 963us/step - loss: 0.5836 - accuracy: 0.6839
Epoch 3/10
130/130 [==============================] - 0s 1ms/step - loss: 0.5818 - accuracy: 0.6864
Epoch 4/10
130/130 [==============================] - 0s 946us/step - loss: 0.5808 - accuracy: 0.6837
Epoch 5/10
130/130 [==============================] - 0s 953us/step - loss: 0.5801 - accuracy: 0.6825
Epoch 6/10
130/130 [==============================] - 0s 1ms/step - loss: 0.5796 - accuracy: 0.6802
Epoch 7/10
130/130 [==============================] - 0s 985us/step - loss: 0.5784 - accuracy: 0.6834
Epoch 8/10
130/130 [==============================] - 0s 1ms/step - loss: 0.5775 - accuracy: 0.6822
Epoch 9/10
130/130 [==============================] - 0s 1ms/step - loss: 0.5775 - accuracy: 0.6825
Epoch 10/10
45/45 [==============================] - 0s 894us/step
Precisão no conjunto de t

In [130]:
result, given_prob, calc_prob = simulate_prediction(dados)

Resultado: ['Sim' 'Nao' 'Nao' ... 'Sim' 'Nao' 'Nao']
Probabilidade fornecida: 0.30157603294050833
Probabilidade calculada: 0.010357477166082728


In [131]:

print("Matriz de Recursos (X):")
print(X)

print("\nVariável Alvo (y):")
print(y)

Matriz de Recursos (X):
[[-0.38048514]
 [-0.34199446]
 [-1.12484389]
 ...
 [-0.07915234]
 [-1.04132307]
 [ 1.01050352]]

Variável Alvo (y):
0       Nao
1       Nao
2       Sim
3       Nao
4       Sim
       ... 
7038    Nao
7039    Nao
7040    Nao
7041    Sim
7042    Nao
Name: Churn, Length: 7043, dtype: object


In [132]:
#XGBoost
@optimize_xgboost_decorator
def churn_prediction_xgboost(X, y):
    return X, y


model = churn_prediction_xgboost(X, y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


Precisão no conjunto de teste: 0.8410220014194464

Relatório de Classificação:
              precision    recall  f1-score   support

         Nao       0.91      0.87      0.89      1036
         Sim       0.68      0.76      0.72       373

    accuracy                           0.84      1409
   macro avg       0.79      0.81      0.80      1409
weighted avg       0.85      0.84      0.84      1409


Exemplos de resultados reais, previstos e probabilidades:
     Real Previsto  Probabilidade_Pos
185   Sim      Sim           0.798301
2715  Nao      Sim           0.567472
3825  Nao      Nao           0.059448


In [136]:
result, given_prob, calc_prob = simulate_prediction(dados)

Resultado: ['Nao' 'Sim' 'Sim' ... 'Nao' 'Sim' 'Nao']
Probabilidade fornecida: 0.2984523640494108
Probabilidade calculada: 0.010388329676964042


In [133]:
@preprocess_data_decorator
def database_preprocessing(df):
    return df

dados = pd.read_csv(
    path,sep=",",encoding="latin-1")
X, y = database_preprocessing(dados)



In [134]:
df_filtered.head(4)

,Maior65Anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,VariasLinhasTelefonicas,ServicoDeInternet,SegurancaOnline,BackupOnline,SeguroNoDispositivo,SuporteTecnico,TVaCabo,StreamingDeFilmes,TipoDeContrato,PagamentoOnline,FormaDePagamento,ContaMensal,Churn
0,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.85,Nao
1,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,ChequePapel,56.95,Nao
2,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,53.85,Sim
3,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30,Nao


In [138]:
#Percorre todo o dataframe, verifica se existem valores nulos
#trata com col.fillna(0) substitui por 0, verifica o tipo se é object "Valor nao numerico e se for ja converte com OneHotEncoder"

df_filtered = dados.apply(lambda col: col.fillna(0) if col.name in [col for col in dados.columns if dados[col].dtype == 'object'] else col)



In [139]:
df_filtered

,Maior65Anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,VariasLinhasTelefonicas,ServicoDeInternet,SegurancaOnline,BackupOnline,SeguroNoDispositivo,SuporteTecnico,TVaCabo,StreamingDeFilmes,TipoDeContrato,PagamentoOnline,FormaDePagamento,ContaMensal,Churn
0,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.85,Nao
1,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,ChequePapel,56.95,Nao
2,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,53.85,Sim
3,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.30,Nao
4,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,70.70,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,Sim,Sim,24,Sim,Sim,DSL,Sim,Nao,Sim,Sim,Sim,Sim,UmAno,Sim,ChequePapel,84.80,Nao
7039,0,Sim,Sim,72,Sim,Sim,FibraOptica,Nao,Sim,Sim,Nao,Sim,Sim,UmAno,Sim,CartaoDeCredito,103.20,Nao
7040,0,Sim,Sim,11,Nao,SemServicoTelefonico,DSL,Sim,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequeDigital,29.60,Nao
7041,1,Sim,Nao,4,Sim,Sim,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,ChequePapel,74.40,Sim
